In [1]:
import requests
import json
import pandas as pd
import os
import dotenv
import pdfbox
import re
from IPython.display import clear_output
dotenv.load_dotenv()
FDAPI = os.environ["FUND_DAILY_API_KEY"]
FFAPI = os.environ["FUND_FACT_API_KEY"]
global FundDailyKey
FundDailyKey= {
    'Ocp-Apim-Subscription-Key': FDAPI
}
global FundFactKey
FundFactKey = {
    'Ocp-Apim-Subscription-Key': FFAPI
}
project_type_keys = ['proj_retail_type', 'proj_term_flag', 'proj_term_dd', 'proj_term_mm','proj_term_yy']
specification_keys = ['spec_code', 'spec_desc']
suitability_keys = ['risk_spectrum', 'risk_spectrum_desc','fund_suitable_desc']

In [2]:
def FundFactAPIGet(url, proj_unique_id, type, key):
    if type == 'amc':
        response = requests.get(url,headers=key)
        if response.status_code == 200:
            return json.loads(response.text)
    elif type in ['project_type', 'specification', 'suitability']:
        response = requests.get(url + proj_unique_id + '/' + type, headers=key)
        if response.status_code == 200:
            return {'json': json.loads(response.text), 'status_code': response.status_code}
        else:
            dummy_json = {key: None for key in project_type_keys} if type == 'project_type' else ({key: None for key in specification_keys} if type == 'specification' else {key: None for key in suitability_keys})
            return {'json': dummy_json, 'status_code': response.status_code}
    elif type in ['URLs']:
        response = requests.get(url + proj_unique_id + '/'+ type, headers=key)
        res_json = json.loads(response.text)
        res_pdf = requests.get(res_json['url_factsheet'])
        if res_pdf.status_code == 200:
            with open(f"tmp/tmp.pdf", 'wb') as pdf_file:
                    pdf_file.write(res_pdf.content)
            return res_pdf.status_code
        else:
            return res_pdf.status_code

In [3]:
def readRiskFromPdf(risk_level):
    if risk_level != 200:
           return None
    p = pdfbox.PDFBox()
    if os.path.getsize("tmp/tmp.pdf") < 216:
            raise Exception("File not found ::") 
    p.extract_text(f'tmp/tmp.pdf')
    with open(f'tmp/tmp.txt', 'r',  encoding="utf8") as file:
            data = file.read()
    match1 = re.search(r'(\S+)\s*:\s*เสี่ยง', data)
    match2 = re.search(r'(\S+)\s*:\s*เสียง', data)
    match3 = re.search(r'(\S+)\s*:\s*เสยง', data)
    if match1:
            return match1.group(1)
    elif match2:
            return match2.group(1)
    elif match3:
            return match3.group(1)
    else:
            return None

In [4]:
def getPdf(row):
    print(row['proj_id'])
    status_code = FundFactAPIGet('https://api.sec.or.th/FundFactsheet/fund/', row['proj_id'] ,'URLs',FundFactKey)
    risk_level = readRiskFromPdf(status_code)
    print(risk_level)
    return risk_level

In [5]:
df = pd.read_csv(f'../Result_amc/SCB/SCB_funds_clean.csv')
df

,Unnamed: 0.1,Unnamed: 0,proj_id,regis_id,regis_date,cancel_date,proj_name_th,proj_name_en,proj_abbr_name,fund_status,...,proj_retail_type,proj_term_flag,proj_term_dd,proj_term_mm,proj_term_yy,risk_spectrum,risk_spectrum_desc,fund_suitable_desc,spec_code,spec_desc
0,0,0,M0002_2563,MF0023_2563,2020-02-04,-,กองทุนเปิดไทยพาณิชย์ Diversified Income Plus,SCB Diversified Income Plus Fund,SCBDPLUS,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,- ผู้ลงทุนที่สามารถรับความผันผวนของราคาหุ้นที่...,NaN,NaN
1,1,1,M0003_2539,004_2540,1997-02-27,-,กองทุนเปิดไทยพาณิชย์ตราสารหนี้ระยะสั้น,SCB Short Term Fixed Income Fund,SCBSFF,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,<p>- ผู้ลงทุนที่คาดหวังผลตอบแทนที่สูงกว่าเงินฝ...,NaN,NaN
2,2,2,M0004_2564,MF0024_2564,2021-01-26,-,กองทุนเปิดไทยพาณิชย์ Active Allocation,SCB Active Allocation Fund,SCBAA,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,- ผู้ลงทุนที่สามารถรับความผันผวนของราคาหุ้นที่...,FED,กองทุนรวมฟีดเดอร์
3,3,3,M0005_2537,033_2537,1994-09-26,-,กองทุนเปิดไทยพาณิชย์ธนอนันต์,SCB DHANA ANANTA OPEN END FUND,SCBDAFUND,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,- ผู้ลงทุนที่สามารถรับความผันผวนของราคาหุ้นที่...,NaN,NaN
4,4,4,M0005_2558,MF0081_2558,2015-02-13,-,กองทุนเปิดไทยพาณิชย์ หุ้นจีนเอแชร์,SCB CHINA A-SHARES FUND,SCBCHAFUND,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,- ผู้ลงทุนที่สามารถรับความผันผวนของราคาหุ้นที่...,"FED,SSF","กองทุนรวมฟีดเดอร์,กองทุนรวมเพื่อการออม"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,262,262,M0938_2566,MF0029_2567,2024-01-23,-,กองทุนเปิดไทยพาณิชย์ มุ่งรักษาเงินต้น 6 เดือน 34,SCB Capital Protection 6M34,SCBCP6M34,RG,...,R,Y,NaN,6.0,NaN,NaN,NaN,NaN,CRF,กองทุนรวมมุ่งรักษาเงินต้น
218,263,263,M0939_2566,MF0014_2567,2024-01-15,-,กองทุนเปิดไทยพาณิชย์ มุ่งรักษาเงินต้น 6 เดือน 33,SCB Capital Protection 6M33,SCBCP6M33,RG,...,R,Y,NaN,6.0,NaN,NaN,NaN,NaN,CRF,กองทุนรวมมุ่งรักษาเงินต้น
219,264,264,M0940_2566,MF0038_2567,2024-01-29,-,กองทุนเปิดไทยพาณิชย์ มุ่งรักษาเงินต้น 6 เดือน 35,SCB Capital Protection 6M35,SCBCP6M35,RG,...,R,Y,NaN,6.0,NaN,NaN,NaN,NaN,CRF,กองทุนรวมมุ่งรักษาเงินต้น
220,265,265,M0977_2556,MF0405_2557,2014-06-24,-,กองทุนรวมอสังหาริมทรัพย์ปิ่นทอง อินดัสเตรียล ป...,Pinthong Industrial Park Property Fund,PPF,RG,...,R,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df['risk_spectrum'] = df.apply(lambda row : getPdf(row), axis=1)

M0002_2563
5
M0003_2539
4
M0004_2564
5
M0005_2537
6
M0005_2558
6
M0005_2565
5
M0006_2539
4
M0006_2563
5
M0007_2538
6
M0015_2564
6
M0017_2565
6
M0024_2560
6
M0026_2539
6
M0026_2551
6
M0031_2557
6
M0049_2565
6
M0059_2564
6
M0061_2561
5
M0065_2558
6
M0074_2544
4
M0079_2544
4
M0079_2564
6
M0080_2544
5
M0081_2544
6
M0086_2559
6
M0088_2565
6
M0090_2561
6
M0094_2561
5
M0110_2559
5
M0110_2564
6
M0116_2564
5
M0120_2564
7
M0124_2561
6
M0134_2564
6
M0141_2559
6
M0142_2561
6
M0144_2563
6
M0146_2565
6
M0151_2563
5
M0151_2564
5
M0152_2564
6
M0154_2561
4
M0155_2556
6
M0158_2564
6
M0162_2564
6
M0163_2564
6
M0164_2564
7
M0173_2552
4
M0181_2563
6
M0181_2564
6
M0182_2565
6
M0185_2565
6
M0191_2550
6
M0196_2564
6
M0202_2554
None
M0205_2566
3
M0206_2566
3
M0207_2549
6
M0210_2556
None
M0211_2564
7
M0228_2554
6
M0232_2564
7
M0241_2554
4
M0245_2559
6
M0252_2557
None
M0254_2565
6
M0259_2563
5
M0260_2550
6
M0262_2564
5
M0263_2550
6
M0263_2566
None
M0274_2564
6
M0277_2564
6
M0279_2561
None
M0284_2555
4
M0285_2564

In [7]:
df['risk_spectrum'].value_counts().sum()

208

# Check non rmf and ssf distribution

In [8]:
dropna_df = df[df['spec_code'].notna()]
dropna_df['spec_code'].value_counts()

spec_code
CRF            44
FED            28
FED,SSF        28
FED,RMF        18
FF,SSF         13
SSF            11
FF              8
LTF,SSF         6
RMF             4
FED,SE,SSF      3
FF,RMF          3
FED,SE          2
IND,SSF         2
IND,SE          2
TESG            2
FED,RMF,SE      2
MM              2
IND,RMF         1
FF,SE,SSF       1
IND,LTF,SSF     1
LTF             1
SPF             1
IND             1
IND,TESG        1
Name: count, dtype: int64

In [9]:
# non_df = dropna_df[dropna_df['spec_code'].str.contains('RMF')].reset_index()
non_df = dropna_df[~dropna_df['spec_code'].str.contains('RMF|SSF')].reset_index()
print(non_df['spec_code'].value_counts())
print(non_df['risk_spectrum'].value_counts())

spec_code
CRF         44
FED         28
FF           8
MM           2
FED,SE       2
IND,SE       2
TESG         2
IND          1
SPF          1
LTF          1
IND,TESG     1
Name: count, dtype: int64
risk_spectrum
3    39
6    26
5    13
7     4
4     2
1     2
Name: count, dtype: int64


In [11]:
non_df.to_csv('non_url_factsheet.csv')

In [13]:
non_df.head(1)

,index,Unnamed: 0.1,Unnamed: 0,proj_id,regis_id,regis_date,cancel_date,proj_name_th,proj_name_en,proj_abbr_name,...,proj_retail_type,proj_term_flag,proj_term_dd,proj_term_mm,proj_term_yy,risk_spectrum,risk_spectrum_desc,fund_suitable_desc,spec_code,spec_desc
0,2,2,2,M0004_2564,MF0024_2564,2021-01-26,-,กองทุนเปิดไทยพาณิชย์ Active Allocation,SCB Active Allocation Fund,SCBAA,...,R,N,NaN,NaN,NaN,5,NaN,- ผู้ลงทุนที่สามารถรับความผันผวนของราคาหุ้นที่...,FED,กองทุนรวมฟีดเดอร์


In [12]:
slice_df = non_df.iloc[:, 3:]
slice_df

       proj_id     regis_id  regis_date cancel_date  \
0   M0004_2564  MF0024_2564  2021-01-26           -   
1   M0015_2564  MF0049_2564  2021-02-17           -   
2   M0024_2560  MF0136_2560  2017-03-28           -   
3   M0026_2539     045_2539  1996-08-20           -   
4   M0026_2551     026_2551  2008-02-06           -   
..         ...          ...         ...         ...   
87  M0936_2566  MF0018_2567  2024-01-16           -   
88  M0937_2566  MF0044_2567  2024-01-30           -   
89  M0938_2566  MF0029_2567  2024-01-23           -   
90  M0939_2566  MF0014_2567  2024-01-15           -   
91  M0940_2566  MF0038_2567  2024-01-29           -   

                                         proj_name_th  \
0              กองทุนเปิดไทยพาณิชย์ Active Allocation   
1               กองทุนเปิดไทยพาณิชย์ Asia Opportunity   
2   กองทุนเปิดไทยพาณิชย์ ตราสารหนี้ US Short Durat...   
3          กองทุนเปิดไทยพาณิชย์ เซ็ท อินเด็กซ์ ฟันด์    
4   กองทุนเปิดไทยพาณิชย์ เอเชียน อีเมอร์จิ้ง มาร์เ... 

# Check RMF&SSF distribution

In [ ]:
dropna_df = df[df['spec_code'].notna()]
dropna_df['spec_code'].value_counts()

In [ ]:
rmf_df = dropna_df[dropna_df['spec_code'].str.contains('RMF')].reset_index()
print(rmf_df['spec_code'].value_counts())
print(rmf_df['risk_spectrum'].value_counts())

In [ ]:
ssf_df = dropna_df[dropna_df['spec_code'].str.contains('SSF')].reset_index()
print(ssf_df['spec_code'].value_counts())
print(ssf_df['risk_spectrum'].value_counts())